In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
!pip install ray==1.0
#!pip install GPy sklearn

#!pip install torch==1.0.1
#!pip install torchvision==0.3

   


In [3]:
from ray.tune.logger import *
supernombre="PBTHYPERLAST5";
class TestLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join("/content/drive/My Drive/hyper3.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        #if "done" in tmp:
         #   if(tmp["done"] != True):

        if "config" in tmp:
            del tmp["config"]
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            if not self._continuing:
                self._csv_out.writeheader()
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()

from ray.tune import Stopper

class CustomStopper(Stopper):
    def __init__(self):
        self.should_stop = False

    def __call__(self, trial_id, result):
        if not self.should_stop and result['loss'] > 2000:
          return True
        else:
          return False

    def stop_all(self):
        """Returns whether to stop trials and prevent new ones from starting."""
        return False


In [2]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10, STL10
import torchvision.transforms as transforms
import os
import torchvision.transforms as T

from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.suggest import ConcurrencyLimiter
from ray import tune

globalinc=0
#ray.memory

kwargs = {'num_workers': 1, 'pin_memory': False} if torch.cuda.is_available() else {}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batchSize = 512
imageSize = 32
data_transform = transforms.Compose([
                transforms.ToTensor()])
path = '/home/antoine/Projet/data1/'
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('/home/antoine/Projet/data1/', train=True, download=True,
                   transform=data_transform),
        batch_size=batchSize, shuffle=True, **kwargs)


test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('/home/antoine/Projet/data1/', train=False, download=True,
                   transform=data_transform),
        batch_size=batchSize, shuffle=True, **kwargs) 

from ray.tune.utils import pin_in_object_store, get_pinned_object
import ray

ray.shutdown()
ray.init()

X_id = pin_in_object_store(train_loader) 
Y_id = pin_in_object_store(test_loader)


Files already downloaded and verified
Files already downloaded and verified


2020-12-13 18:56:49,984	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265


In [ ]:
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
# Transformations
RC   = transforms.RandomCrop(32, padding=4)
RHF  = transforms.RandomHorizontalFlip()
RVF  = transforms.RandomVerticalFlip()
NRM  = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
TT   = transforms.ToTensor()
TPIL = transforms.ToPILImage()

# Transforms object for trainset with augmentation
#transform_with_aug = transforms.Compose([TPIL, RC, RHF, TT, NRM])
# Transforms object for testset with NO augmentation
transform_no_aug   = transforms.Compose([TT])
path = '/home/antoine/Projet/data1/'

# Downloading/Louding CIFAR10 data
trainset  = CIFAR10(root=path, train=True , download=True)#, transform = transform_with_aug)
testset   = CIFAR10(root=path, train=False, download=True)#, transform = transform_no_aug)
classDict = {'plane':0, 'car':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}

# Separating trainset/testset data/label
x_train = trainset.data
x_test = testset.data
y_train = trainset.targets
y_test = testset.targets

# Define a function to separate CIFAR classes by class index

def get_class_i(x, y, i):
    """
    x: trainset.train_data or testset.test_data
    y: trainset.train_labels or testset.test_labels
    i: class label, a number between 0 to 9
    test_loaderreturn: x_i
    """
    # Convert to a numpy array
    y = np.array(y)
    # Locate position of labels that equal to i
    pos_i = np.argwhere(y == i)
    # Convert the result into a 1-D list
    pos_i = list(pos_i[:,0])
    # Collect all data that match the desired label
    x_i = [x[j] for j in pos_i]
    
    return x_i

class DatasetMaker(Dataset):
    def __init__(self, datasets, transformFunc = transform_no_aug):
        """
        datasets: a list of get_class_i outputs, i.e. a list of list of images for selected classes
        """
        self.datasets = datasets
        self.lengths  = [len(d) for d in self.datasets]
        self.transformFunc = transformFunc
    def __getitem__(self, i):
        class_label, index_wrt_class = self.index_of_which_bin(self.lengths, i)
        img = self.datasets[class_label][index_wrt_class]
     #   img = Image.fromarray(img)
        img = self.transformFunc(img)
        return img, class_label

    def __len__(self):
        return sum(self.lengths)
    
    def index_of_which_bin(self, bin_sizes, absolute_index, verbose=False):
        """
        Given the absolute index, returns which bin it falls in and which element of that bin it corresponds to.
        """
        # Which class/bin does i fall into?
        accum = np.add.accumulate(bin_sizes)
        if verbose:
            print("accum =", accum)
        bin_index  = len(np.argwhere(accum <= absolute_index))
        if verbose:
            print("class_label =", bin_index)
        # Which element of the fallent class/bin does i correspond to?
        index_wrt_class = absolute_index - np.insert(accum, 0, 0)[bin_index]
        if verbose:
            print("index_wrt_class =", index_wrt_class)

        return bin_index, index_wrt_class

# ================== Usage ================== #

# Let's choose cats (class 3 of CIFAR) and dogs (class 5 of CIFAR) as trainset/testset
cat_dog_trainset = \
    DatasetMaker(
        [get_class_i(x_train, y_train, classDict['dog'])],
        transform_no_aug
    )
cat_dog_testset  = \
    DatasetMaker(
        [ get_class_i(x_test , y_test , classDict['dog'])],
        transform_no_aug
    )

kwargs = {'num_workers': 2, 'pin_memory': False}

batchSize = 512
imageSize = 32

# Create datasetLoaders from trainset and testset
trainsetLoader   = DataLoader(cat_dog_trainset, batch_size=batchSize, shuffle=True , **kwargs)
testsetLoader    = DataLoader(cat_dog_testset , batch_size=batchSize, shuffle=False, **kwargs)

from ray.tune.utils import pin_in_object_store, get_pinned_object
import ray

ray.shutdown()
ray.init()

X_id = pin_in_object_store(trainsetLoader) 
Y_id = pin_in_object_store(testsetLoader)

Files already downloaded and verified
Files already downloaded and verified


2020-12-12 00:51:40,250	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


In [4]:
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F

#Residual down sampling block for the encoder
#Average pooling is used to perform the downsampling
class Res_down(nn.Module):
    def __init__(self, channel_in, channel_out, scale = 2):
        super(Res_down, self).__init__()
        
        self.conv1 = nn.Conv2d(channel_in, channel_out//2, 3, 1, 1)
        self.BN1 = nn.BatchNorm2d(channel_out//2)
        self.conv2 = nn.Conv2d(channel_out//2, channel_out, 3, 1, 1)
        self.BN2 = nn.BatchNorm2d(channel_out)
        
        self.conv3 = nn.Conv2d(channel_in, channel_out, 3, 1, 1)

        self.AvePool = nn.AvgPool2d(scale,scale)
        
    def forward(self, x):
        skip = self.conv3(self.AvePool(x))
        
        x = F.rrelu(self.BN1(self.conv1(x)))
        x = self.AvePool(x)
        x = self.BN2(self.conv2(x))
        
        x = F.rrelu(x + skip)
        return x

    
#Residual up sampling block for the decoder
#Nearest neighbour is used to perform the upsampling
class Res_up(nn.Module):
    def __init__(self, channel_in, channel_out, scale = 2):
        super(Res_up, self).__init__()
        
        self.conv1 = nn.Conv2d(channel_in, channel_out//2, 3, 1, 1)
        self.BN1 = nn.BatchNorm2d(channel_out//2)
        self.conv2 = nn.Conv2d(channel_out//2, channel_out, 3, 1, 1)
        self.BN2 = nn.BatchNorm2d(channel_out)
        
        self.conv3 = nn.Conv2d(channel_in, channel_out, 3, 1, 1)
        
        self.UpNN = nn.Upsample(scale_factor = scale,mode = "nearest")
        
    def forward(self, x):
        skip = self.conv3(self.UpNN(x))
        
        x = F.rrelu(self.BN1(self.conv1(x)))
        x = self.UpNN(x)
        x = self.BN2(self.conv2(x))
        
        x = F.rrelu(x + skip)
        return x
    
#Encoder block
#Built for a 64x64x3 image and will result in a latent vector of size Z x 1 x 1 
#As the network is fully convolutional it will work for other larger images sized 2^n the latent
#feature map size will just no longer be 1 - aka Z x H x W
class Encoder(nn.Module):
    def __init__(self, channels, ch = 32, z = 128*4):
        super(Encoder, self).__init__()
        self.conv1 = Res_down(channels, ch)#64
        self.conv2 = Res_down(ch, 2*ch)#32
        self.conv3 = Res_down(2*ch, 4*ch)#16
        self.conv4 = Res_down(4*ch, 8*ch)#8
        self.conv5 = Res_down(8*ch, 16*ch)#4
        self.conv_mu = nn.Conv2d(16*ch, z, 1, 1)#2
        self.conv_logvar = nn.Conv2d(16*ch, z, 1, 1)#2 //TODO

    def sample(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
        
    def forward(self, x, Train = True):
        batch_size=x.shape[0]
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)

        if Train:
            mu = self.conv_mu(x)
            logvar = self.conv_logvar(x)
            x = self.sample(mu, logvar)

        else:
            x = self.conv_mu(x)
            mu = None
            logvar = None
        return x.reshape(batch_size,-1), mu.reshape(batch_size,-1), logvar.reshape(batch_size,-1)
    
#Decoder block
#Built to be a mirror of the encoder block
class Decoder(nn.Module):
    def __init__(self, channels, ch = 32, z = 128*4):
        super(Decoder, self).__init__()

        self.conv1 = Res_up(z, ch*8)
        self.conv2 = Res_up(ch*8, ch*8)
        self.conv3 = Res_up(ch*8, ch*4)
        self.conv4 = Res_up(ch*4, ch*2)
        self.conv5 = Res_up(ch*2, ch)
        self.conv6 = Res_up(ch, ch)
        self.conv7 = nn.Conv2d(ch, channels, 3, 1, 1)

    def forward(self, x):
        x=x.reshape(x.shape[0],-1,1,1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)

        #x = self.conv6(x)
        x = self.conv7(x)
        return x.reshape(x.shape[0],-1) 
    
#VAE network, uses the above encoder and decoder blocks 
class VAES(nn.Module):
    def __init__(self, channel_in=3, z = 128*4,prob=0):
        super(VAES, self).__init__()
        """Res VAE Network
        channel_in  = number of channels of the image 
        z = the number of channels of the latent representation (for a 64x64 image this is the size of the latent vector)"""
        self.d = nn.Dropout(prob)

        self.encoder = Encoder(channel_in, z = z)
        self.decoder = Decoder(channel_in, z = z)

    def forward(self, x, Train = True):
        encoding, mu, logvar = self.encoder(self.d(x), Train)
        recon = self.decoder(self.d(encoding))
        return recon, mu, logvar
        

In [ ]:

from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10, STL10
import torchvision.transforms as transforms
import os
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

def train_VAE(config):
    parser = argparse.ArgumentParser(description='VAE MNIST Example')
    parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                        help='input batch size for training (default: 128)')
    parser.add_argument('--epochs', type=int, default=training, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    args, _ = parser.parse_known_args()
    args.cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if args.cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': False} if args.cuda else {}
    #train_loader = torch.utils.data.DataLoader(
    #    datasets.MNIST('../data', train=True, download=True,
    #                   transform=transforms.ToTensor()),
    #    batch_size=args.batch_size, shuffle=True, **kwargs)
    #test_loader = torch.utils.data.DataLoader(
    #    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    #batch_size=args.batch_size, shuffle=True, **kwargs)



    class VAE(nn.Module):
        def __init__(self):
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            super(VAE, self).__init__()

            dim1= int(config.get("d1", 0)) #36*12*4*3
            dim2=int(config.get("d2", 0)) #36*12*3
            dim3=int(config.get("d3", 0)) #20*27
            dim4=int(config.get("d4", 0)) #dim2
            dim5=int(config.get("d5", 0)) #dim1
            prob=config.get("dropout",0)

           # self.fc0 = nn.Linear(32*32*3,400*3)
          #  self.fc1 = nn.Linear(784*3, 400*3)
          #  self.fc21 = nn.Linear(400*3, 20*3)
          #  self.fc22 = nn.Linear(400*3, 20*3)
          #  self.fc3 = nn.Linear(20*3, 400*3)
           # self.fc4 = nn.Linear(400*3, 784*3)
           # self.fc5 = nn.Linear(400*3,32*32*3)
            
            self.lf=cst*cst*3

            self.fc1 = nn.Linear(self.lf, dim1)

            self.fc12 = nn.Linear(dim1, dim2)

            self.fc21 = nn.Linear(dim2, dim3)
            self.fc22 = nn.Linear(dim2, dim3)

            self.fc23 = nn.Linear(dim3,dim4)

            self.fc3 = nn.Linear(dim4, dim5)

            self.fc4 = nn.Linear(dim5, self.lf)
            self.d = nn.Dropout(prob)

        def encoder(self, x):
            h1 = F.relu(self.fc1(x.view(-1, 32*32*3)))
            h1=self.d(h1)
            h1 = F.relu(self.fc12(h1))
            h1=torch.sigmoid(self.d(h1))
            return self.fc21(h1), self.fc22(h1)

        def reparameterize(self, mu, logvar):
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return mu + eps*std

        def decoder(self, z):
            h3 = F.relu(self.fc23(z))
            h3=self.d(h3)
            h3 = F.relu(self.fc3(h3))
            h3=self.d(h3)
            return torch.sigmoid(self.fc4(h3))

        def forward(self, x):
            mu, logvar = self.encoder(x)
            z = self.reparameterize(mu, logvar)
            return self.decoder(z), mu, logvar


    #model = VAE().to(device)
    model = VAES(channel_in = 3,prob=config.get("dropout",0)).to(device)
    #optimizer = adabelief_pytorch.AdaBelief(model.parameters(), lr=1e-3)
    lr=10**-(config.get("lr", 0))
    beta1=1-10**-(config.get("b1", 0))
    beta2=1-10**-(config.get("b2", 0))
    eps=10**-(config.get("eps", 0))
    weight_decay=10**-(config.get("wd", 0))
    optimizer = optim.Adam(model.parameters()#, lr=1e-3)
                            ,  lr=lr, betas=(beta1, beta2),
                           eps=eps, weight_decay=weight_decay)
    #print(optimizer)
    
    
    
    
    # Reconstruction + KL divergence losses summed over all elements and batch
    def loss_function(recon_x, x, mu, logvar):
        BCE = F.binary_cross_entropy_with_logits(recon_x, x.view(-1, cst*cst*3), reduction='sum')

        # see Appendix B from VAE paper:
        # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
        # https://arxiv.org/abs/1312.6114
        # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return BCE + KLD


    def train(train_loader):
        model.train()
        train_loss = 0
        model.to(device)
        for i in range(1):
          train_loss = 0
          for batch_idx, (data, _) in enumerate(train_loader):
        #     if(batch_idx>10):
          #        break
              data = data.to(device)
              optimizer.zero_grad()
              recon_batch, mu, logvar = model(data)
              loss = loss_function(recon_batch, data, mu, logvar)
              loss.backward()
              train_loss += loss.item()
              optimizer.step()
        print(len(train_loader.dataset))
        print(train_loss / len(train_loader.dataset))


    def test(test_loader):
        model.eval()
        test_loss = 0
        model.to(device)
        with torch.no_grad():
            for i, (data, _) in enumerate(test_loader):
           #     if(i>10):
            #        break
                data = data.to(device)
                recon_batch, mu, logvar = model(data)
                test_loss += loss_function(recon_batch, data, mu, logvar).item()
                #if i == 0:
                #    n = min(data.size(0), 8)
                #    comparison = torch.cat([data[:n],
                #                          recon_batch.view(batchSize, 3, cst, cst)[:n]])
                #    save_image(comparison.cpu(),
                #             path+ 'reconstruction_' + str(os.getpid()) + '.png', nrow=n)

        test_loss /= len(test_loader.dataset)
        tune.report(loss=test_loss)
        #print('====> Test set loss: {:.4f}'.format(test_loss))

        


    if __name__ == "__main__":
        train_loader=  get_pinned_object(X_id)
        test_loader=  get_pinned_object(Y_id)

        #test_loader = train_loader #get_pinned_object(Y_id)
        for epoch in range(1, args.epochs + 1):
            train(train_loader)
            test(test_loader)
           # with torch.no_grad():
           #     sample = torch.randn(64,512).to(device)
           #     sample = model.decoder(sample).cpu()
           #     save_image(sample.view(64, 3, cst, cst),path +'sample_' + str(os.getpid()) + '.png')
        del train_loader
        del test_loader
    
    




from ray.tune.suggest import ConcurrencyLimiter

algo = HyperOptSearch(metric="loss",
    mode="min")
name="hypher"

algo = ConcurrencyLimiter(algo, max_concurrent=2)
#os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
scheduler = ASHAScheduler()


imageSize = 32

stopper = CustomStopper()
training=3
cst=imageSize
total=cst*cst*3*2
def trial_name_id(trial):
    return f"{trial.trainable_name}_{trial.trial_id}"
from ray import tune
analysis = tune.run(
train_VAE,
name=name,
scheduler=scheduler,
reuse_actors=True,
search_alg=algo,
verbose=2,
checkpoint_at_end=False,
num_samples=512,
# export_formats=[ExportFormat.MODEL],
            config= {
 "lr":  tune.uniform(2.5, 8) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "wd":tune.uniform(1, 10)#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": tune.uniform(0, 4)#,1e-4), #*10 et 0
 ,    "b2": tune.uniform(1, 5)#,1e-4), #*10 et 0
 ,    "eps": tune.uniform(2, 10)#,1e-4), #*10 et 0
 #,    "d1": tune.uniform(total/4, total)
 #,    "d2": tune.uniform(total/16, total/4)
 #,    "d3": tune.uniform(total/64, total/16)
 #,    "d4": tune.uniform(total/16, total/4)
#,    "d5": tune.uniform(total/4, total)
, "dropout":tune.uniform(0,0.2)

                
    },      stop=stopper
        ,        metric="loss",
    mode="min", resources_per_trial={'gpu': 1}

          ,    loggers=[TestLogger]
)
#all_trials = analysis.trials
#checkpoint_paths = [
#    os.path.join(analysis.get_best_checkpoint(t), "checkpoint")
#    for t in all_trials
#] 
#demo_gan(checkpoint_paths)


2020-12-13 18:57:17,626	WARNING function_runner.py:486 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2020-12-13 18:57:17,674	WARNING logger.py:328 -- JsonLogger not provided. The ExperimentAnalysis tool is disabled.


Trial name,status,loc,b1,b2,dropout,eps,lr,wd
train_VAE_056f0fdc,RUNNING,,3.67977,4.75956,0.174952,9.99383,4.90565,9.46077
train_VAE_05704eb0,PENDING,,3.36261,1.77886,0.00291701,4.91868,3.97512,7.05463


(pid=3795) 50000
(pid=3795) 2096.23091625
Result for train_VAE_056f0fdc:
  date: 2020-12-13_18-58-59
  done: true
  experiment_id: 350396a3dd9e469e8b20baba941ae221
  experiment_tag: 1_b1=3.6798,b2=4.7596,dropout=0.17495,eps=9.9938,lr=4.9057,wd=9.4608
  hostname: 7af10844b0f4
  iterations_since_restore: 1
  loss: 2036.08564375
  node_ip: 172.28.0.2
  pid: 3795
  time_since_restore: 100.74668955802917
  time_this_iter_s: 100.74668955802917
  time_total_s: 100.74668955802917
  timestamp: 1607885939
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 056f0fdc
  


Trial name,status,loc,b1,b2,dropout,eps,lr,wd,iter,total time (s),loss
train_VAE_056f0fdc,TERMINATED,,3.67977,4.75956,0.174952,9.99383,4.90565,9.46077,1,100.747,2036.09
train_VAE_05704eb0,PENDING,,3.36261,1.77886,0.00291701,4.91868,3.97512,7.05463,,,


(pid=3796) 50000
(pid=3796) 1987.384835
Result for train_VAE_05704eb0:
  date: 2020-12-13_19-00-41
  done: true
  experiment_id: b9d99c5294704b6eb177945485e3a18c
  experiment_tag: 2_b1=3.3626,b2=1.7789,dropout=0.002917,eps=4.9187,lr=3.9751,wd=7.0546
  hostname: 7af10844b0f4
  iterations_since_restore: 1
  loss: 2024.9652125
  node_ip: 172.28.0.2
  pid: 3796
  time_since_restore: 100.7452940940857
  time_this_iter_s: 100.7452940940857
  time_total_s: 100.7452940940857
  timestamp: 1607886041
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 05704eb0
  


...

Trial name,status,loc,b1,b2,dropout,eps,lr,wd,iter,total time (s),loss
train_VAE_9ff8752a,PENDING,,1.40029,3.13649,0.00781915,2.5011,4.43069,8.83831,,,
train_VAE_ee99278e,RUNNING,172.28.0.2:6784,0.717265,4.50348,0.0167108,3.81831,3.9431,7.65046,1,99.2119,1933.03
train_VAE_056f0fdc,TERMINATED,,3.67977,4.75956,0.174952,9.99383,4.90565,9.46077,1,100.747,2036.09
train_VAE_05704eb0,TERMINATED,,3.36261,1.77886,0.00291701,4.91868,3.97512,7.05463,1,100.745,2024.97
train_VAE_092c158a,TERMINATED,,0.611301,3.61754,0.147349,4.35814,7.01375,3.46231,1,100.952,2319.13
train_VAE_0b0ee8b0,TERMINATED,,1.19439,1.76166,0.155884,7.05573,4.73582,4.92496,1,100.465,1982.45
train_VAE_0c6f5536,TERMINATED,,1.15443,3.74322,0.0194903,9.85867,2.81579,9.9808,1,100.449,1960.46
train_VAE_10b4a78a,TERMINATED,,0.772859,2.43243,0.000736046,8.67215,3.1493,8.55162,3,297.629,1924.05
train_VAE_15602812,TERMINATED,,0.72736,3.38474,0.145041,5.66106,4.23852,3.02037,3,297.176,1942.95
train_VAE_19966c60,TERMINATED,,1.94928,1.29568,0.071438,9.90033,4.07264,3.84647,1,100.971,2018.84


(pid=6784) 50000
(pid=6784) 1920.0496675
Result for train_VAE_ee99278e:
  date: 2020-12-13_22-03-51
  done: false
  experiment_id: f574521de60e4224af4c5d9b1f3e54aa
  experiment_tag: 68_b1=0.71726,b2=4.5035,dropout=0.016711,eps=3.8183,lr=3.9431,wd=7.6505
  hostname: 7af10844b0f4
  iterations_since_restore: 2
  loss: 1919.26958125
  node_ip: 172.28.0.2
  pid: 6784
  time_since_restore: 195.74853134155273
  time_this_iter_s: 96.53666043281555
  time_total_s: 195.74853134155273
  timestamp: 1607897031
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: ee99278e
  


Trial name,status,loc,b1,b2,dropout,eps,lr,wd,iter,total time (s),loss
train_VAE_9ff8752a,PENDING,,1.40029,3.13649,0.00781915,2.5011,4.43069,8.83831,,,
train_VAE_ee99278e,RUNNING,172.28.0.2:6784,0.717265,4.50348,0.0167108,3.81831,3.9431,7.65046,2,195.749,1919.27
train_VAE_056f0fdc,TERMINATED,,3.67977,4.75956,0.174952,9.99383,4.90565,9.46077,1,100.747,2036.09
train_VAE_05704eb0,TERMINATED,,3.36261,1.77886,0.00291701,4.91868,3.97512,7.05463,1,100.745,2024.97
train_VAE_092c158a,TERMINATED,,0.611301,3.61754,0.147349,4.35814,7.01375,3.46231,1,100.952,2319.13
train_VAE_0b0ee8b0,TERMINATED,,1.19439,1.76166,0.155884,7.05573,4.73582,4.92496,1,100.465,1982.45
train_VAE_0c6f5536,TERMINATED,,1.15443,3.74322,0.0194903,9.85867,2.81579,9.9808,1,100.449,1960.46
train_VAE_10b4a78a,TERMINATED,,0.772859,2.43243,0.000736046,8.67215,3.1493,8.55162,3,297.629,1924.05
train_VAE_15602812,TERMINATED,,0.72736,3.38474,0.145041,5.66106,4.23852,3.02037,3,297.176,1942.95
train_VAE_19966c60,TERMINATED,,1.94928,1.29568,0.071438,9.90033,4.07264,3.84647,1,100.971,2018.84


(pid=6784) 50000
(pid=6784) 1912.733485
Result for train_VAE_ee99278e:
  date: 2020-12-13_22-05-28
  done: false
  experiment_id: f574521de60e4224af4c5d9b1f3e54aa
  experiment_tag: 68_b1=0.71726,b2=4.5035,dropout=0.016711,eps=3.8183,lr=3.9431,wd=7.6505
  hostname: 7af10844b0f4
  iterations_since_restore: 3
  loss: 1914.8102625
  node_ip: 172.28.0.2
  pid: 6784
  time_since_restore: 292.1567134857178
  time_this_iter_s: 96.40818214416504
  time_total_s: 292.1567134857178
  timestamp: 1607897128
  timesteps_since_restore: 0
  training_iteration: 3
  trial_id: ee99278e
  


Trial name,status,loc,b1,b2,dropout,eps,lr,wd,iter,total time (s),loss
train_VAE_9ff8752a,PENDING,,1.40029,3.13649,0.00781915,2.5011,4.43069,8.83831,,,
train_VAE_ee99278e,RUNNING,172.28.0.2:6784,0.717265,4.50348,0.0167108,3.81831,3.9431,7.65046,3,292.157,1914.81
train_VAE_056f0fdc,TERMINATED,,3.67977,4.75956,0.174952,9.99383,4.90565,9.46077,1,100.747,2036.09
train_VAE_05704eb0,TERMINATED,,3.36261,1.77886,0.00291701,4.91868,3.97512,7.05463,1,100.745,2024.97
train_VAE_092c158a,TERMINATED,,0.611301,3.61754,0.147349,4.35814,7.01375,3.46231,1,100.952,2319.13
train_VAE_0b0ee8b0,TERMINATED,,1.19439,1.76166,0.155884,7.05573,4.73582,4.92496,1,100.465,1982.45
train_VAE_0c6f5536,TERMINATED,,1.15443,3.74322,0.0194903,9.85867,2.81579,9.9808,1,100.449,1960.46
train_VAE_10b4a78a,TERMINATED,,0.772859,2.43243,0.000736046,8.67215,3.1493,8.55162,3,297.629,1924.05
train_VAE_15602812,TERMINATED,,0.72736,3.38474,0.145041,5.66106,4.23852,3.02037,3,297.176,1942.95
train_VAE_19966c60,TERMINATED,,1.94928,1.29568,0.071438,9.90033,4.07264,3.84647,1,100.971,2018.84


In [ ]:

from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10, STL10
import torchvision.transforms as transforms
import os
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.suggest import ConcurrencyLimiter
from ray import tune
import gc
import numpy as np
from ray.tune.schedulers.pb2 import PB2


#train_loader = torch.utils.data.DataLoader(
#    datasets.MNIST('../data', train=True, download=True,
#                   transform=transforms.ToTensor()),
#    batch_size=args.batch_size, shuffle=True, **kwargs)
#test_loader = torch.utils.data.DataLoader(
#    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
#batch_size=args.batch_size, shuffle=True, **kwargs)

class VAE(nn.Module):
    def __init__(self,config):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        super(VAE, self).__init__()
        dim1= int(config.get("d1", 0)) #36*12*4*3
        dim2=int(config.get("d2", 0)) #36*12*3
        dim3=int(config.get("d3", 0)) #20*27
        dim4=int(config.get("d4", 0)) #dim2
        dim5=int(config.get("d5", 0)) #dim1

        print("gang")
        self.lf=image_L*image_L*3
        print("endan")

        self.fc1 = nn.Linear(self.lf, dim1)
        print("endan")

        self.fc12 = nn.Linear(dim1, dim2)
        print("endan")

        self.fc21 = nn.Linear(dim2, dim3)
        print("endan")
        self.fc22 = nn.Linear(dim2, dim3)
        print("endan")

        self.fc23 = nn.Linear(dim3,dim4)

        print("endan")
        self.fc3 = nn.Linear(dim4, dim5)
        print("endan")

        self.fc4 = nn.Linear(dim5, self.lf)
        print("endan")

    def encode(self, x):
        h1 = F.relu(self.fc1(x.view(-1, self.lf)))
        h1 = F.relu(self.fc12(h1))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc23(z))
        h3 = F.relu(self.fc3(h3))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

class PytorchTrainable(tune.Trainable):

    def setup(self, config):
        use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if use_cuda else "cpu")

        #self.netD = VAES(channel_in=3).to(self.device)
        self.vae_net = VAES(channel_in = 3).to(self.device)


        lr=10**-(config.get("lr", 0))
        beta1=1-10**-(config.get("b1", 0))
        beta2=1-10**-(config.get("b2", 0))
        eps=10**-(config.get("eps", 0))
        weight_decay=10**-(config.get("wd", 0))
        self.optimizerD = optim.Adam(self.vae_net.parameters(),
                                  lr=lr, betas=(beta1, beta2),
                               eps=eps, weight_decay=weight_decay)
        self.dimbotle=128*4
        globalinc=0

        self.train_loader=  get_pinned_object(X_id)
        self.test_loader=  get_pinned_object(Y_id)




        #kwargs = {'num_workers': 1, 'pin_memory': False} if args.cuda else {}
       # self.train_loader = torch.utils.data.DataLoader(
        #    datasets.STL10('/content/drive/My Drive/', split="unlabeled", download=True,
        #                   transform=transforms.ToTensor()),
        #        batch_size=args.batch_size, shuffle=True, **kwargs)
        #self.test_loader = torch.utils.data.DataLoader(
        #    datasets.STL10('/content/drive/My Drive/', split="test", transform=transforms.ToTensor(),download=True),
        #batch_size=args.batch_size, shuffle=True, **kwargs)


    def step(self):
        device = torch.device("cuda" if  torch.cuda.is_available() else "cpu")
        train(self.vae_net,self.optimizerD,self.train_loader)
        loss=test(self.vae_net,self.test_loader)
        with torch.no_grad():
            sample = torch.randn(64,self.dimbotle).to(device)
            sample = self.vae_net.decoder(sample).cpu()
            save_image(sample.view(64, 3, cst, cst),path +'sample_' + str(os.getpid()) + '.png')

    
        return {"loss": loss}

    def save_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        torch.save({
            "netDmodel": self.vae_net.state_dict(),
            "optimD": self.optimizerD.state_dict(),

        }, path)

        return checkpoint_dir

    def load_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        checkpoint = torch.load(path)
        self.vae_net.load_state_dict(checkpoint["netDmodel"])
        self.optimizerD.load_state_dict(checkpoint["optimD"])

    def reset_config(self, new_config):
        if "lr" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["lr"] = 10**-(new_config.get("lr", 0))
        if "b1" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["betas"] = (
                    1-10**-(new_config.get("b1", 0)),1-10**-(new_config.get("b2", 0)))
        if "wd" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["weight_decay"] = 10**-(new_config.get("wd", 0))
        if "eps" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["eps"] = 10**-(new_config.get("eps", 0))
        self.config = new_config
        return True



#  model = VAE().to(device)
#optimizer = adabelief_pytorch.AdaBelief(model.parameters(), lr=1e-3)





# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy_with_logits(recon_x, x.view(-1, cst*cst*3), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(model,optimizer,train_loader):
    model.train()
    train_loss = 0
    device = torch.device("cuda" if  torch.cuda.is_available() else "cpu")
    model.to(device)
    for i in range(1):
          train_loss = 0
          for batch_idx, (data, _) in enumerate(train_loader):
          #   if(batch_idx>10):
            #      break
              data = data.to(device)
              optimizer.zero_grad()
              recon_batch, mu, logvar = model(data)
              loss = loss_function(recon_batch, data, mu, logvar)
              loss.backward()
              train_loss += loss.item()
              optimizer.step()
              del data
       # if batch_idx % args.log_interval == 0:
       #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
       #         epoch, batch_idx * len(data), len(train_loader.dataset),
       #         100. * batch_idx / len(train_loader),
       #         loss.item() / len(data)))

    #print('====> Epoch: {} Average loss: {:.4f}'.format(
  #        epoch, train_loss / len(train_loader.dataset)))


def test(model,test_loader):
    model.eval()
    test_loss = 0
    device = torch.device("cuda" if   torch.cuda.is_available() else "cpu")
    model.to(device)
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
          #  if(i>10):
           #     break
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batchSize, 3, 32, 32)[:n]])
                save_image(comparison.cpu(),
                             path+ 'reconstruction_' + str(os.getpid()) + '.png', nrow=n)
            del data
    test_loss /= len(test_loader.dataset)
    gc.collect()
  #  tune.report(loss=test_loss)
 #   print('====> Test set loss: {:.4f}'.format(test_loss))
    return test_loss

#   if __name__ == "__main__":
#      for epoch in range(1, args.epochs + 1):
  #      train(epoch)
 #       test(epoch)
#        with torch.no_grad():
#             sample = torch.randn(64,int(config.get("d3", 0))).to(device)
#              sample = model.decode(sample).cpu()
#               save_image(sample.view(64, 3, 96, 96),'/home/antoine/Projet/NovelTuning/sample_' + str(epoch) + '.png')







algo = HyperOptSearch(metric="loss",
    mode="min")
name="hypher"

#algo = ConcurrencyLimiter(algo, max_concurrent=1)
#os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
scheduler = PB2(
    time_attr="training_iteration",
    perturbation_interval=3,
    hyperparam_bounds={
        # distribution for resampling
     "lr": [1, 8] #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "wd":[1, 10]#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": [0, 2]#,1e-4), #*10 et 0
 ,    "b2": [1, 5] #,1e-4), #*10 et 0
 ,    "eps": [2, 10]#,1e-4), #*10 et 0
    }) 

#   hyperparam_bounds={
#        # distribution for resampling
#     "lr":  lambda: np.random.uniform(1, 8) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
#,     "wd":lambda: np.random.uniform(1, 10)#tune.uniform(1, 5)#,1e-4), #*10 et 0
#,     "b1": lambda: np.random.uniform(0, 2)#,1e-4), #*10 et 0
# ,    "b2": lambda: np.random.uniform(1, 5)#,1e-4), #*10 et 0
# ,    "eps": lambda: np.random.uniform(2, 10)#,1e-4), #*10 et 0
#    })

imageSize = 32

cst=imageSize
total=cst*cst*3
def trial_name_id(trial):
    return f"{trial.trainable_name}_{trial.trial_id}"
from ray import tune

analysis = tune.run(
PytorchTrainable,
name=name,
scheduler=scheduler,
reuse_actors=True,
search_alg=algo,
verbose=2,
checkpoint_at_end=True,
num_samples=1024,
# export_formats=[ExportFormat.MODEL],
            config= {
 "lr":  tune.uniform(2.5, 8) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "wd":tune.uniform(1, 10)#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": tune.uniform(0, 2)#,1e-4), #*10 et 0
 ,    "b2": tune.uniform(1, 5)#,1e-4), #*10 et 0
 ,    "eps": tune.uniform(2, 10)#,1e-4), #*10 et 0
# ,    "d1": tune.uniform(27648/6, 27648/3)
# ,    "d2": tune.uniform(27648/12, 27648/6)
# ,    "d3": tune.uniform(27648/24, 27648/12)
# ,    "d4": tune.uniform(27648/12, 27648/6)
#,    "d5": tune.uniform(27648/6, 27648/3)

                
    },      stop={
        "training_iteration": 10,
    },        metric="loss",
    mode="min",resources_per_trial={'gpu': 1}
              loggers=[TestLogger])
)
#all_trials = analysis.trials
#checkpoint_paths = [
#    os.path.join(analysis.get_best_checkpoint(t), "checkpoint")
#    for t in all_trials
#] 
#demo_gan(checkpoint_paths) in


2020-12-12 00:51:49,705	INFO logger.py:201 -- pip install 'ray[tune]' to see TensorBoard files.
2020-12-12 00:51:49,706	WARNING logger.py:343 -- Could not instantiate TBXLogger: No module named 'tensorboardX'.


Trial name,status,loc,b1,b2,eps,lr,wd
PytorchTrainable_37bb0d72,RUNNING,,1.69036,4.74776,5.774,4.0995,6.71466


Result for PytorchTrainable_37c701d6:
  date: 2020-12-12_00-55-23
  done: false
  experiment_id: b3b383e90d4543a0ae27c24678db8d9e
  experiment_tag: 2_b1=1.1852,b2=2.6307,eps=3.2016,lr=4.6917,wd=9.6191
  hostname: 18986fe86903
  iterations_since_restore: 1
  loss: 2126.815375
  node_ip: 172.28.0.2
  pid: 1767
  time_since_restore: 210.84373092651367
  time_this_iter_s: 210.84373092651367
  time_total_s: 210.84373092651367
  timestamp: 1607734523
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 37c701d6
  


Trial name,status,loc,b1,b2,eps,lr,wd,iter,total time (s),loss
PytorchTrainable_37bb0d72,RUNNING,,1.69036,4.74776,5.774,4.0995,6.71466,,,
PytorchTrainable_37c701d6,RUNNING,172.28.0.2:1767,1.18519,2.6307,3.20157,4.69173,9.61909,1,210.844,2126.82
PytorchTrainable_37cb9eb2,PENDING,,0.662954,2.17069,6.86938,6.09436,4.48347,,,


Result for PytorchTrainable_37bb0d72:
  date: 2020-12-12_00-55-25
  done: false
  experiment_id: f2dcac73d72a4f8889da7ed6175ab8a7
  experiment_tag: 1_b1=1.6904,b2=4.7478,eps=5.774,lr=4.0995,wd=6.7147
  hostname: 18986fe86903
  iterations_since_restore: 1
  loss: 2122.02925
  node_ip: 172.28.0.2
  pid: 1766
  time_since_restore: 212.52030873298645
  time_this_iter_s: 212.52030873298645
  time_total_s: 212.52030873298645
  timestamp: 1607734525
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 37bb0d72
  
Result for PytorchTrainable_37c701d6:
  date: 2020-12-12_00-58-50
  done: false
  experiment_id: b3b383e90d4543a0ae27c24678db8d9e
  experiment_tag: 2_b1=1.1852,b2=2.6307,eps=3.2016,lr=4.6917,wd=9.6191
  hostname: 18986fe86903
  iterations_since_restore: 2
  loss: 2124.9190625
  node_ip: 172.28.0.2
  pid: 1767
  time_since_restore: 418.45657539367676
  time_this_iter_s: 207.6128444671631
  time_total_s: 418.45657539367676
  timestamp: 1607734730
  timesteps_since_restore: 

...

Result for PytorchTrainable_9f8e0180:
  date: 2020-12-12_03-47-51
  done: true
  experiment_id: f2dcac73d72a4f8889da7ed6175ab8a7
  experiment_tag: 9_b1=1.6586,b2=4.5546,eps=7.9198,lr=7.294,wd=5.0867@perturbed[b1=1.6904,b2=4.7478,eps=5.774,lr=4.0995,wd=6.7147]
  hostname: 18986fe86903
  iterations_since_restore: 1
  loss: 1966.6761875
  node_ip: 172.28.0.2
  pid: 2575
  time_since_restore: 209.86902952194214
  time_this_iter_s: 209.86902952194214
  time_total_s: 2091.9075965881348
  timestamp: 1607744871
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 9f8e0180
  


Trial name,status,loc,b1,b2,eps,lr,wd,iter,total time (s),loss
PytorchTrainable_9f8e0180,RUNNING,172.28.0.2:2575,1.69036,4.74776,5.774,4.0995,6.71466,10,2091.91,1966.68
PytorchTrainable_1ec64be6,RUNNING,,0,1,10,1,1,9,1892.78,1981.01
PytorchTrainable_37bb0d72,TERMINATED,,1.69036,4.74776,5.774,4.0995,6.71466,10,2097.9,1971.81
PytorchTrainable_37c701d6,TERMINATED,,1.18519,2.6307,3.20157,4.69173,9.61909,10,2092.35,1986.11
PytorchTrainable_37cb9eb2,TERMINATED,,1.4795,4.00094,5.23819,3.67673,8.35631,10,2087.16,1967.31
PytorchTrainable_ae28c930,TERMINATED,,1.4795,4.00094,5.23819,3.67673,8.35631,10,2087.25,1961.88
PytorchTrainable_aebbf20a,TERMINATED,,1.4795,4.00094,5.23819,3.67673,8.35631,10,2104.02,1963.13
PytorchTrainable_2620a2a4,TERMINATED,,1.4795,4.00094,5.23819,3.67673,8.35631,10,2102.26,1977.63
PytorchTrainable_264a859c,TERMINATED,,0.91148,3.50544,2.2017,3.79387,2.41239,10,2112.07,1940.6
PytorchTrainable_9f052d38,TERMINATED,,0.222869,3.43505,2,1.67252,1,10,2090.03,1969.42


Result for PytorchTrainable_1ec64be6:
  date: 2020-12-12_03-47-52
  done: true
  experiment_id: d39f5fa19cbd46b0a120e5bb71d6c700
  experiment_tag: 10_b1=0.052328,b2=3.7109,eps=2.1143,lr=3.4007,wd=7.0318@perturbed[b1=0.0,b2=1.0,eps=10.0,lr=1.0,wd=1.0]
  hostname: 18986fe86903
  iterations_since_restore: 1
  loss: 1941.4579375
  node_ip: 172.28.0.2
  pid: 2576
  time_since_restore: 208.28950667381287
  time_this_iter_s: 208.28950667381287
  time_total_s: 2109.489803314209
  timestamp: 1607744872
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 1ec64be6
  


Trial name,status,loc,b1,b2,eps,lr,wd,iter,total time (s),loss
PytorchTrainable_37bb0d72,TERMINATED,,1.69036,4.74776,5.774,4.0995,6.71466,10,2097.9,1971.81
PytorchTrainable_37c701d6,TERMINATED,,1.18519,2.6307,3.20157,4.69173,9.61909,10,2092.35,1986.11
PytorchTrainable_37cb9eb2,TERMINATED,,1.4795,4.00094,5.23819,3.67673,8.35631,10,2087.16,1967.31
PytorchTrainable_ae28c930,TERMINATED,,1.4795,4.00094,5.23819,3.67673,8.35631,10,2087.25,1961.88
PytorchTrainable_aebbf20a,TERMINATED,,1.4795,4.00094,5.23819,3.67673,8.35631,10,2104.02,1963.13
PytorchTrainable_2620a2a4,TERMINATED,,1.4795,4.00094,5.23819,3.67673,8.35631,10,2102.26,1977.63
PytorchTrainable_264a859c,TERMINATED,,0.91148,3.50544,2.2017,3.79387,2.41239,10,2112.07,1940.6
PytorchTrainable_9f052d38,TERMINATED,,0.222869,3.43505,2,1.67252,1,10,2090.03,1969.42
PytorchTrainable_9f8e0180,TERMINATED,,1.69036,4.74776,5.774,4.0995,6.71466,10,2091.91,1966.68
PytorchTrainable_1ec64be6,TERMINATED,,0,1,10,1,1,10,2109.49,1941.46


2020-12-12 03:47:52,801	INFO tune.py:439 -- Total run time: 10563.13 seconds (10563.05 seconds for the tuning loop).


In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10, STL10
import torchvision.transforms as transforms
import os
import torchvision.transforms as T

from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.suggest import ConcurrencyLimiter
from ray import tune
#def train_VAE(config):
parser = argparse.ArgumentParser(description='VAE MNIST Example')
parser.add_argument('--batch-size', type=int, default=512, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args, _ = parser.parse_known_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if args.cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': False} if args.cuda else {}
#train_loader = torch.utils.data.DataLoader(
#    datasets.MNIST('../data', train=True, download=True,
#                   transform=transforms.ToTensor()),
#    batch_size=args.batch_size, shuffle=True, **kwargs)
#test_loader = torch.utils.data.DataLoader(
#    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
#batch_size=args.batch_size, shuffle=True, **kwargs)

data_transform = transforms.Compose([
                transforms.ToTensor()])
transform = T.Compose([T.Resize(64), T.ToTensor()])

# datasets

#train_loader = torch.utils.data.DataLoader(
#    datasets.STL10('/home/antoine/Projet/data', split="unlabeled", target_transform=transform,download=True),
#batch_size=args.batch_size, shuffle=True, **kwargs)

kwargs = {'num_workers': 1, 'pin_memory': False} if args.cuda else {}
batchSize=512
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('/home/antoine/Projet/data1/', train=True, download=True,
                   transform=data_transform),
        batch_size=batchSize, shuffle=True, **kwargs)


test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('/home/antoine/Projet/data1/', train=False, download=True,
                   transform=data_transform),
        batch_size=batchSize, shuffle=True, **kwargs) 
#train_loader=test_loader;



In [ ]:
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F

#Residual down sampling block for the encoder
#Average pooling is used to perform the downsampling
class Res_down(nn.Module):
    def __init__(self, channel_in, channel_out, scale = 2):
        super(Res_down, self).__init__()
        
        self.conv1 = nn.Conv2d(channel_in, channel_out//2, 3, 1, 1)
        self.BN1 = nn.BatchNorm2d(channel_out//2)
        self.conv2 = nn.Conv2d(channel_out//2, channel_out, 3, 1, 1)
        self.BN2 = nn.BatchNorm2d(channel_out)
        
        self.conv3 = nn.Conv2d(channel_in, channel_out, 3, 1, 1)

        self.AvePool = nn.AvgPool2d(scale,scale)
        
    def forward(self, x):
        skip = self.conv3(self.AvePool(x))
        
        x = F.rrelu(self.BN1(self.conv1(x)))
        x = self.AvePool(x)
        x = self.BN2(self.conv2(x))
        
        x = F.rrelu(x + skip)
        return x

    
#Residual up sampling block for the decoder
#Nearest neighbour is used to perform the upsampling
class Res_up(nn.Module):
    def __init__(self, channel_in, channel_out, scale = 2):
        super(Res_up, self).__init__()
        
        self.conv1 = nn.Conv2d(channel_in, channel_out//2, 3, 1, 1)
        self.BN1 = nn.BatchNorm2d(channel_out//2)
        self.conv2 = nn.Conv2d(channel_out//2, channel_out, 3, 1, 1)
        self.BN2 = nn.BatchNorm2d(channel_out)
        
        self.conv3 = nn.Conv2d(channel_in, channel_out, 3, 1, 1)
        
        self.UpNN = nn.Upsample(scale_factor = scale,mode = "nearest")
        
    def forward(self, x):
        skip = self.conv3(self.UpNN(x))
        
        x = F.rrelu(self.BN1(self.conv1(x)))
        x = self.UpNN(x)
        x = self.BN2(self.conv2(x))
        
        x = F.rrelu(x + skip)
        return x
    
#Encoder block
#Built for a 64x64x3 image and will result in a latent vector of size Z x 1 x 1 
#As the network is fully convolutional it will work for other larger images sized 2^n the latent
#feature map size will just no longer be 1 - aka Z x H x W
class Encoder(nn.Module):
    def __init__(self, channels, ch = 32, z = 128):
        super(Encoder, self).__init__()
        self.conv1 = Res_down(channels, ch)#64
        self.conv2 = Res_down(ch, 2*ch)#32
        self.conv3 = Res_down(2*ch, 4*ch)#16
        self.conv4 = Res_down(4*ch, 8*ch)#8
        self.conv5 = Res_down(8*ch, 8*ch)#4
        self.conv_mu = nn.Conv2d(8*ch, z, 2, 2)#2
        self.conv_logvar = nn.Conv2d(8*ch, z, 2, 2)#2

    def sample(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
        
    def forward(self, x, Train = True):
        batch_size=x.shape[0]
        print(batch_size)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)

        if Train:
            mu = self.conv_mu(x)
            logvar = self.conv_logvar(x)
            x = self.sample(mu, logvar)
        else:
            x = self.conv_mu(x)
            mu = None
            logvar = None
        return x.reshape(batch_size,-1), mu.reshape(batch_size,-1), logvar.reshape(batch_size,-1)
    
#Decoder block
#Built to be a mirror of the encoder block
class Decoder(nn.Module):
    def __init__(self, channels, ch = 32, z = 128):
        super(Decoder, self).__init__()
        self.conv1 = Res_up(z, ch*8)
        self.conv2 = Res_up(ch*8, ch*8)
        self.conv3 = Res_up(ch*8, ch*4)
        self.conv4 = Res_up(ch*4, ch*2)
        self.conv5 = Res_up(ch*2, ch)
        self.conv6 = Res_up(ch*2, ch//2)
        self.conv7 = nn.Conv2d(ch//2, channels, 3, 1, 1)

    def forward(self, x):
        print("hell")
        print(x.shape[0])
        x=x.reshape(x.shape[0],-1,1,1)
        
        x = self.conv1(x)
        print(x.shape)

        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        #x = self.conv5(x)
        x = self.conv6(x)
        print(x.shape)

        x = self.conv7(x)
        print(x.shape)
        print("end")
        return x.reshape(x.shape[0],-1) 
    
#VAE network, uses the above encoder and decoder blocks 
class VAES(nn.Module):
    def __init__(self, channel_in, z = 128):
        super(VAES, self).__init__()
        """Res VAE Network
        channel_in  = number of channels of the image 
        z = the number of channels of the latent representation (for a 64x64 image this is the size of the latent vector)"""
        
        self.encoder = Encoder(channel_in, z = z)
        self.decoder = Decoder(channel_in, z = z)

    def forward(self, x, Train = True):
        print("hello" ,x.shape)
        encoding, mu, logvar = self.encoder(x, Train)
        print("encoding", encoding.shape)
        recon = self.decoder(encoding)
        print(recon.shape)
        return recon, mu, logvar

In [ ]:

#test = FashionFashionMNIST(root="./data", train=False, transform=data_transform, download=True)
#train_loader = torch.utils.data.DataLoader(train, batch_size=args.batch_size, shuffle=True, num_workers=0)
#test_loader = torch.utils.data.DataLoader(test, batch_size=args.batch_size, shuffle=True, num_workers=0)
#ray.shutdown();
#ray.init()
class VAE(nn.Module):
    def __init__(self,config):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        super(VAE, self).__init__()

        dim1= int(config.get("d1", 0)) #36*12*4*3
        dim2=int(config.get("d2", 0)) #36*12*3
        dim3=int(config.get("d3", 0)) #20*27
        dim4=int(config.get("d4", 0)) #dim2
        dim5=int(config.get("d5", 0)) #dim1

        self.lf=12288

        self.fc1 = nn.Linear(self.lf, dim1)

        self.fc12 = nn.Linear(dim1, dim2)

        self.fc21 = nn.Linear(dim2, dim3)
        self.fc22 = nn.Linear(dim2, dim3)

        self.fc23 = nn.Linear(dim3,dim4)

        self.fc3 = nn.Linear(dim4, dim5)

        self.fc4 = nn.Linear(dim5, self.lf)

    def encode(self, x):
        h1 = F.relu(self.fc1(x.view(-1, self.lf)))
        h1 = F.relu(self.fc12(h1))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc23(z))
        h3 = F.relu(self.fc3(h3))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        #print("Hello",x.shape)
        mu, logvar = self.encode(x)
        #print("encoding",mu.shape)
        z = self.reparameterize(mu, logvar)
        
        de=self.decode(z)
        return de, mu, logvar

class PytorchTrainable(tune.Trainable):

    def setup(self, config):
       # print(get_pinned_object(X_id).shape)
        use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if use_cuda else "cpu")
        self.netD = VAE(config).to(self.device)
        lr=10**-(config.get("lr", 0))
        beta1=1-10**-(config.get("b1", 0))
        beta2=1-10**-(config.get("b2", 0))
        eps=10**-(config.get("eps", 0))
        weight_decay=10**-(config.get("weight_decay", 0))
        self.optimizerD = optim.Adam(self.netD.parameters(),
                                  lr=lr, betas=(beta1, beta2),
                               eps=eps, weight_decay=weight_decay)
        self.dimbotle=int(config.get("d3", 0))
        self.vae_net = VAES(channel_in = 3).to(device)
    def step(self):
        global globalinc
     #   for epoch in range(1, args.epochs + 1):
        #print(self.optimizerD)
        train(self.vae_net,self.optimizerD)
        loss=test(self.vae_net)
        with torch.no_grad():
            sample = torch.randn(64,self.dimbotle).to(device)
            sample = self.netD.decode(sample).cpu()
            save_image(sample.view(64, 3, 64, 64),'/home/antoine/Projet/NovelTuning/sample_' + str(globalinc) + '.png')
            globalinc= globalinc+1

        return {"loss": loss}

    def save_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        torch.save({
            "netDmodel": self.netD.state_dict(),
            "optimD": self.optimizerD.state_dict(),

        }, path)

        return checkpoint_dir

    def load_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        checkpoint = torch.load(path)
        self.netD.load_state_dict(checkpoint["netDmodel"])
        self.optimizerD.load_state_dict(checkpoint["optimD"])

    def reset_config(self, new_config):
        if "lr" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["lr"] = 10**-(new_config.get("lr", 0))
        if "b1" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["betas"] = (
                    1-10**-(new_config.get("b1", 0)),1-10**-(new_config.get("b2", 0)))
        if "wd" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["weight_decay"] = 10**-(new_config.get("wd", 0))
        if "eps" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["eps"] = 10**-(new_config.get("eps", 0))
        self.config = new_config
        return True



#  model = VAE().to(device)
#optimizer = adabelief_pytorch.AdaBelief(model.parameters(), lr=1e-3)





# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy_with_logits(recon_x, x.view(-1, 32*32*3), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(model,optimizer):
    model.train()
    train_loss = 0
    train_loader = get_pinned_object(X_id)
    for batch_idx, (data, _) in enumerate(train_loader):
        #print("coucou")
        #if(batch_idx>10):
        #    break
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        #print(recon_batch.shape, mu.shape, logvar.shape)
        #return
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
       # if batch_idx % args.log_interval == 0:
       #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
       #         epoch, batch_idx * len(data), len(train_loader.dataset),
       #         100. * batch_idx / len(train_loader),
       #         loss.item() / len(data)))

    #print('====> Epoch: {} Average loss: {:.4f}'.format(
  #        epoch, train_loss / len(train_loader.dataset)))


def test(model):
    model.eval()
    test_loss = 0
   # test_loader=get_pinned_object(X_id)
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            #if(i>10):
            #    break
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(args.batch_size, 3, 64, 64)[:n]])
                save_image(comparison.cpu(),
                         '/home/antoine/Projet/NovelTuning/reconstruction_' + str(globalinc) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
  #  tune.report(loss=test_loss)
 #   print('====> Test set loss: {:.4f}'.format(test_loss))
    return test_loss

#   if __name__ == "__main__":
#      for epoch in range(1, args.epochs + 1):
  #      train(epoch)
 #       test(epoch)
#        with torch.no_grad():
#             sample = torch.randn(64,int(config.get("d3", 0))).to(device)
#              sample = model.decode(sample).cpu()
#               save_image(sample.view(64, 3, 96, 96),'/home/antoine/Projet/NovelTuning/sample_' + str(epoch) + '.png')







algo = HyperOptSearch(metric="loss",
    mode="min")
name="hypher"

algo = ConcurrencyLimiter(algo, max_concurrent=1)
#os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    perturbation_interval=1,
    hyperparam_mutations={
        # distribution for resampling
     "lr":  lambda: np.random.uniform(1, 8) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "weight_decay":lambda: np.random.uniform(1, 10)#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": lambda: np.random.uniform(0, 2)#,1e-4), #*10 et 0
 ,    "b2": lambda: np.random.uniform(1, 5)#,1e-4), #*10 et 0
 ,    "eps": lambda: np.random.uniform(2, 10)#,1e-4), #*10 et 0
    })

def trial_name_id(trial):
    return f"{trial.trainable_name}_{trial.trial_id}"


from ray.tune.utils import pin_in_object_store, get_pinned_object
import ray
ray.shutdown()
ray.init()

X_id = pin_in_object_store(train_loader)


analysis = tune.run(

PytorchTrainable,
name=name,
scheduler=scheduler,
reuse_actors=True,
search_alg=algo,
verbose=2,
checkpoint_at_end=True,
num_samples=1,
# export_formats=[ExportFormat.MODEL],
            config= {
 "lr":  tune.uniform(1, 8) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "weight_decay":tune.uniform(1, 10)#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": tune.uniform(0, 2)#,1e-4), #*10 et 0
 ,    "b2": tune.uniform(1, 5)#,1e-4), #*10 et 0
 ,    "eps": tune.uniform(2, 10)#,1e-4), #*10 et 0
 ,    "d1": tune.uniform(27648/16, 27648/2)
 ,    "d2": tune.uniform(27648/128, 27648/16)
 ,    "d3": tune.uniform(27648/516, 27648/128)
 ,    "d4": tune.uniform(27648/64, 27648/16)
,    "d5": tune.uniform(27648/8, 27648/2)

                
    },      stop={
        "training_iteration": 3,
    },        metric="loss",
    mode="min", resources_per_trial={'gpu': 1}
 #             loggers=[TestLogger])
)
#all_trials = analysis.trials
#checkpoint_paths = [
#    os.path.join(analysis.get_best_checkpoint(t), "checkpoint")
#    for t in all_trials
#] 
#demo_gan(checkpoint_paths)


In [ ]:
!pip install ray==1.0.0
#!pip install sqlalchemy==1.3.19 
#!pip install tensorboardX==2.1


In [ ]:

batchsize = 128
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10, STL10
import torchvision.transforms as transforms
import os
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.suggest import ConcurrencyLimiter
from ray import tune


In [ ]:
#kwargs = {'num_workers': 1, 'pin_memory': False} if  torch.cuda.is_available() else {}
#
#train_loader = torch.utils.data.DataLoader(
#            datasets.STL10('/content/drive/My Drive/', split="unlabeled", download=True,
#                           transform=transforms.ToTensor()),
#                batch_size=128, shuffle=True, **kwargs)
#test_loader = torch.utils.data.DataLoader(
#            datasets.STL10('/content/drive/My Drive/', split="test", transform=transforms.ToTensor(),download=True),
#        batch_size=128, shuffle=True, **kwargs)
import torchvision.transforms as T


transform = T.Compose([T.Resize(64), T.ToTensor()])

# datasets
batchsize = 128

image_L=64

#train_loader = torch.utils.data.DataLoader(
#    datasets.STL10('/home/antoine/Projet/data', split="unlabeled", target_transform=transform,download=True),
#batch_size=args.batch_size, shuffle=True, **kwargs)

kwargs = {'num_workers': 1, 'pin_memory': False} if torch.cuda.is_available() else {}

#train_loader = torch.utils.data.DataLoader(
#    datasets.STL10('/content/drive/My Drive/', split="unlabeled", download=True,
#                   transform=transform),
#        batch_size=batchsize, shuffle=True, **kwargs)


test_loader = torch.utils.data.DataLoader(
    datasets.STL10('/content/drive/My Drive/', split="test", transform=transform,download=True),
batch_size=batchsize, shuffle=True, **kwargs)

In [ ]:
#kwargs = {'num_workers': 1, 'pin_memory': False} if  torch.cuda.is_available() else {}
#
#train_loader = torch.utils.data.DataLoader(
#            datasets.STL10('/content/drive/My Drive/', split="unlabeled", download=True,
#                           transform=transforms.ToTensor()),
#                batch_size=128, shuffle=True, **kwargs)
#test_loader = torch.utils.data.DataLoader(
#            datasets.STL10('/content/drive/My Drive/', split="test", transform=transforms.ToTensor(),download=True),
#        batch_size=128, shuffle=True, **kwargs)
#import torchvision.transforms as T


#transform = T.Compose([T.Resize(64), T.ToTensor()])

# datasets
#batchsize = 128

image_L=32


from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10, STL10
import torchvision.transforms as transforms
import os
import torchvision.transforms as T

from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.suggest import ConcurrencyLimiter
from ray import tune

globalinc=0
#ray.memory

kwargs = {'num_workers': 1, 'pin_memory': False} if torch.cuda.is_available() else {}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batchSize = 128
imageSize = 32
data_transform = transforms.Compose([
                transforms.ToTensor()])
path = 'home/antoine/Projet/data1'
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('/home/antoine/Projet/data1', train=True, download=True,
                   transform=data_transform),
        batch_size=batchSize, shuffle=True, **kwargs)


test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('/home/antoine/Projet/data1', train=False, download=True,
                   transform=data_transform),
        batch_size=batchSize, shuffle=True, **kwargs) 

from ray.tune.utils import pin_in_object_store, get_pinned_object
import ray

ray.shutdown()
ray.init()

X_id = pin_in_object_store(train_loader) 
Y_id = pin_in_object_store(test_loader)


In [ ]:

from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10, STL10
import torchvision.transforms as transforms
import os
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.suggest import ConcurrencyLimiter
from ray import tune
import gc
import numpy as np

#train_loader = torch.utils.data.DataLoader(
#    datasets.MNIST('../data', train=True, download=True,
#                   transform=transforms.ToTensor()),
#    batch_size=args.batch_size, shuffle=True, **kwargs)
#test_loader = torch.utils.data.DataLoader(
#    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
#batch_size=args.batch_size, shuffle=True, **kwargs)

class VAE(nn.Module):
    def __init__(self,config):
        print("a")
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("b")
        super(VAE, self).__init__()
        print("c")
        dim1= int(config.get("d1", 0)) #36*12*4*3
        dim2=int(config.get("d2", 0)) #36*12*3
        dim3=int(config.get("d3", 0)) #20*27
        dim4=int(config.get("d4", 0)) #dim2
        dim5=int(config.get("d5", 0)) #dim1

        print("gang")
        self.lf=image_L*image_L*3
        print("endan")

        self.fc1 = nn.Linear(self.lf, dim1)
        print("endan")

        self.fc12 = nn.Linear(dim1, dim2)
        print("endan")

        self.fc21 = nn.Linear(dim2, dim3)
        print("endan")
        self.fc22 = nn.Linear(dim2, dim3)
        print("endan")

        self.fc23 = nn.Linear(dim3,dim4)

        print("endan")
        self.fc3 = nn.Linear(dim4, dim5)
        print("endan")

        self.fc4 = nn.Linear(dim5, self.lf)
        print("endan")

    def encode(self, x):
        h1 = F.relu(self.fc1(x.view(-1, self.lf)))
        h1 = F.relu(self.fc12(h1))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc23(z))
        h3 = F.relu(self.fc3(h3))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

class PytorchTrainable(tune.Trainable):

    def setup(self, config):
        print("hello")
        use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if use_cuda else "cpu")
        print("H")

        self.netD = VAE(config).to(self.device)
        print("hello2")

        lr=10**-(config.get("lr", 0))
        beta1=1-10**-(config.get("b1", 0))
        beta2=1-10**-(config.get("b2", 0))
        eps=10**-(config.get("eps", 0))
        weight_decay=10**-(config.get("weight_decay", 0))
        self.optimizerD = optim.Adam(self.netD.parameters(),
                                  lr=lr, betas=(beta1, beta2),
                               eps=eps, weight_decay=weight_decay)
        self.dimbotle=int(config.get("d3", 0))
        globalinc=0
        print("some alloc")

        #def train_VAE(config):
        parser = argparse.ArgumentParser(description='VAE MNIST Example')
        parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                            help='input batch size for training (default: 128)')
        parser.add_argument('--epochs', type=int, default=10, metavar='N',
                            help='number of epochs to train (default: 10)')
        parser.add_argument('--no-cuda', action='store_true', default=False,
                            help='disables CUDA training')
        parser.add_argument('--seed', type=int, default=1, metavar='S',
                            help='random seed (default: 1)')
        parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                            help='how many batches to wait before logging training status')
        args, _ = parser.parse_known_args()
        args.cuda = not args.no_cuda and torch.cuda.is_available()
        self.args=args
        #data_transform = transforms.Compose([
           #             transforms.ToTensor()
           #     ])
        # datasets


        #torch.manual_seed(args.seed)

        device = torch.device("cuda" if args.cuda else "cpu")
        print("end")

        #kwargs = {'num_workers': 1, 'pin_memory': False} if args.cuda else {}
       # self.train_loader = torch.utils.data.DataLoader(
        #    datasets.STL10('/content/drive/My Drive/', split="unlabeled", download=True,
        #                   transform=transforms.ToTensor()),
        #        batch_size=args.batch_size, shuffle=True, **kwargs)
        #self.test_loader = torch.utils.data.DataLoader(
        #    datasets.STL10('/content/drive/My Drive/', split="test", transform=transforms.ToTensor(),download=True),
        #batch_size=args.batch_size, shuffle=True, **kwargs)


    def step(self):
        global globalinc
     #   for epoch in range(1, args.epochs + 1):
        print(self.optimizerD)
        train(self.netD,self.optimizerD)
        loss=test(self.netD)
        with torch.no_grad():
            sample = torch.randn(64,self.dimbotle).to('cuda')
            sample = self.netD.decode(sample).cpu()
            save_image(sample.view(64, 3, image_L, image_L),'/'+path+ '/sample_' + str(globalinc) + '.png')
            globalinc= globalinc+1

        return {"loss": loss}

    def save_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        torch.save({
            "netDmodel": self.netD.state_dict(),
            "optimD": self.optimizerD.state_dict(),

        }, path)

        return checkpoint_dir

    def load_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        checkpoint = torch.load(path)
        self.netD.load_state_dict(checkpoint["netDmodel"])
        self.optimizerD.load_state_dict(checkpoint["optimD"])

    def reset_config(self, new_config):
        if "lr" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["lr"] = 10**-(new_config.get("lr", 0))
        if "b1" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["betas"] = (
                    1-10**-(new_config.get("b1", 0)),1-10**-(new_config.get("b2", 0)))
        if "wd" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["weight_decay"] = 10**-(new_config.get("wd", 0))
        if "eps" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["eps"] = 10**-(new_config.get("eps", 0))
        self.config = new_config
        return True



#  model = VAE().to(device)
#optimizer = adabelief_pytorch.AdaBelief(model.parameters(), lr=1e-3)





# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, image_L*image_L*3), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(model,optimizer):
    model.train()
    train_loss = 0
    device = torch.device("cuda" if  torch.cuda.is_available() else "cpu")
    model.to(device)
    for batch_idx, (data, _) in enumerate(train_loader):
     #   if(batch_idx>10):
      #      break
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        del data
       # if batch_idx % args.log_interval == 0:
       #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
       #         epoch, batch_idx * len(data), len(train_loader.dataset),
       #         100. * batch_idx / len(train_loader),
       #         loss.item() / len(data)))

    #print('====> Epoch: {} Average loss: {:.4f}'.format(
  #        epoch, train_loss / len(train_loader.dataset)))


def test(model):
    model.eval()
    test_loss = 0
    device = torch.device("cuda" if   torch.cuda.is_available() else "cpu")
    
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
          #  if(i>10):
           #     break
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batchSize, 3, image_L, image_L)[:n]])
                save_image(comparison.cpu(),
                         '/'+path+ '/reconstruction_' + str(globalinc) + '.png', nrow=n)
            del data
    test_loss /= len(test_loader.dataset)
    gc.collect()
  #  tune.report(loss=test_loss)
 #   print('====> Test set loss: {:.4f}'.format(test_loss))
    return test_loss

#   if __name__ == "__main__":
#      for epoch in range(1, args.epochs + 1):
  #      train(epoch)
 #       test(epoch)
#        with torch.no_grad():
#             sample = torch.randn(64,int(config.get("d3", 0))).to(device)
#              sample = model.decode(sample).cpu()
#               save_image(sample.view(64, 3, 96, 96),'/home/antoine/Projet/NovelTuning/sample_' + str(epoch) + '.png')







algo = HyperOptSearch(metric="loss",
    mode="min")
name="hypher"

#algo = ConcurrencyLimiter(algo, max_concurrent=1)
#os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    perturbation_interval=1,
    hyperparam_mutations={
        # distribution for resampling
     "lr":  lambda: np.random.uniform(1, 8) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "weight_decay":lambda: np.random.uniform(1, 10)#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": lambda: np.random.uniform(0, 2)#,1e-4), #*10 et 0
 ,    "b2": lambda: np.random.uniform(1, 5)#,1e-4), #*10 et 0
 ,    "eps": lambda: np.random.uniform(2, 10)#,1e-4), #*10 et 0
    })

def trial_name_id(trial):
    return f"{trial.trainable_name}_{trial.trial_id}"



analysis = tune.run(
PytorchTrainable,
name=name,
scheduler=scheduler,
reuse_actors=False,
search_alg=algo,
verbose=2,
checkpoint_at_end=False,
num_samples=2,
# export_formats=[ExportFormat.MODEL],
            config= {
 "lr":  tune.uniform(2.5, 8) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "weight_decay":tune.uniform(1, 10)#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": tune.uniform(0, 2)#,1e-4), #*10 et 0
 ,    "b2": tune.uniform(1, 5)#,1e-4), #*10 et 0
 ,    "eps": tune.uniform(2, 10)#,1e-4), #*10 et 0
 ,    "d1": tune.uniform(27648/6, 27648/3)
 ,    "d2": tune.uniform(27648/12, 27648/6)
 ,    "d3": tune.uniform(27648/24, 27648/12)
 ,    "d4": tune.uniform(27648/12, 27648/6)
,    "d5": tune.uniform(27648/6, 27648/3)

                
    },      stop={
        "training_iteration": 50,
    },        metric="loss",
    mode="min",resources_per_trial={'gpu': 1}
 #             loggers=[TestLogger])
)
#all_trials = analysis.trials
#checkpoint_paths = [
#    os.path.join(analysis.get_best_checkpoint(t), "checkpoint")
#    for t in all_trials
#] 
#demo_gan(checkpoint_paths)


In [ ]:

from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10, STL10
import torchvision.transforms as transforms
import os
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

def train_VAE(config):
    parser = argparse.ArgumentParser(description='VAE MNIST Example')
    parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                        help='input batch size for training (default: 128)')
    parser.add_argument('--epochs', type=int, default=10, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    args, _ = parser.parse_known_args()
    args.cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if args.cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': False} if args.cuda else {}
    #train_loader = torch.utils.data.DataLoader(
    #    datasets.MNIST('../data', train=True, download=True,
    #                   transform=transforms.ToTensor()),
    #    batch_size=args.batch_size, shuffle=True, **kwargs)
    #test_loader = torch.utils.data.DataLoader(
    #    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    #batch_size=args.batch_size, shuffle=True, **kwargs)

    data_transform = transforms.Compose([
                    transforms.ToTensor()
            ])
    # datasets

    #train_loader=test_loader
    #test = FashionFashionMNIST(root="./data", train=False, transform=data_transform, download=True)
    #train_loader = torch.utils.data.DataLoader(train, batch_size=args.batch_size, shuffle=True, num_workers=0)
    #test_loader = torch.utils.data.DataLoader(test, batch_size=args.batch_size, shuffle=True, num_workers=0)

    class VAE(nn.Module):
        def __init__(self):
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            super(VAE, self).__init__()

            dim1= int(config.get("d1", 0)) #36*12*4*3
            dim2=int(config.get("d2", 0)) #36*12*3
            dim3=int(config.get("d3", 0)) #20*27
            dim4=int(config.get("d4", 0)) #dim2
            dim5=int(config.get("d5", 0)) #dim1

            self.lf=27648

            self.fc1 = nn.Linear(self.lf, dim1)

            self.fc12 = nn.Linear(dim1, dim2)

            self.fc21 = nn.Linear(dim2, dim3)
            self.fc22 = nn.Linear(dim2, dim3)

            self.fc23 = nn.Linear(dim3,dim4)

            self.fc3 = nn.Linear(dim4, dim5)

            self.fc4 = nn.Linear(dim5, self.lf)

        def encode(self, x):
            h1 = F.relu(self.fc1(x.view(-1, self.lf)))
            h1 = F.relu(self.fc12(h1))
            return self.fc21(h1), self.fc22(h1)

        def reparameterize(self, mu, logvar):
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return mu + eps*std

        def decode(self, z):
            h3 = F.relu(self.fc23(z))
            h3 = F.relu(self.fc3(h3))
            return torch.sigmoid(self.fc4(h3))

        def forward(self, x):
            mu, logvar = self.encode(x)
            z = self.reparameterize(mu, logvar)
            return self.decode(z), mu, logvar


    model = VAE().to(device)
    #optimizer = adabelief_pytorch.AdaBelief(model.parameters(), lr=1e-3)
    lr=10**-(config.get("lr", 0))
    beta1=1-10**-(config.get("b1", 0))
    beta2=1-10**-(config.get("b2", 0))
    eps=10**-(config.get("eps", 0))
    weight_decay=10**-(config.get("wd", 0))
    optimizer = optim.Adam(model.parameters(),
                              lr=lr, betas=(beta1, beta2),
                           eps=eps, weight_decay=weight_decay)
    print(optimizer)
    
    
    
    
    # Reconstruction + KL divergence losses summed over all elements and batch
    def loss_function(recon_x, x, mu, logvar):
        BCE = F.binary_cross_entropy(recon_x, x.view(-1, 27648), reduction='sum')

        # see Appendix B from VAE paper:
        # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
        # https://arxiv.org/abs/1312.6114
        # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return BCE + KLD


    def train(epoch):
        model.train()
        train_loss = 0
        for batch_idx, (data, _) in enumerate(train_loader):
            if(batch_idx>10):
                break
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)
            loss = loss_function(recon_batch, data, mu, logvar)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()


        return epoch, train_loss / len(self.train_loader.dataset)


    def test(epoch):
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for i, (data, _) in enumerate(test_loader):
                if(i>10):
                    break
                data = data.to(device)
                recon_batch, mu, logvar = model(data)
                test_loss += loss_function(recon_batch, data, mu, logvar).item()
                if i == 0:
                    n = min(data.size(0), 8)
                    comparison = torch.cat([data[:n],
                                          recon_batch.view(args.batch_size, 3, 96, 96)[:n]])
                    save_image(comparison.cpu(),
                             '/home/antoine/Projet/data/'+ 'reconstruction_' + str(epoch) + '.png', nrow=n)

        test_loss /= len(self.test_loader.dataset)
        tune.report(loss=test_loss)
        print('====> Test set loss: {:.4f}'.format(test_loss))

        


    if __name__ == "__main__":
        train_loader=  get_pinned_object(X_id)

        test_loader = get_pinned_object(Y_id)
        for epoch in range(1, args.epochs + 1):
            train(epoch,train_loader)
            test(epoch,test_loader)
            with torch.no_grad():
                sample = torch.randn(64,int(config.get("d3", 0))).to(device)
                sample = model.decode(sample).cpu()
                save_image(sample.view(64, 3, 96, 96),'/home/antoine/Projet/data/' +'sample_' + str(epoch) + '.png')
        del train_loader
        del test_loader
    
    




from ray.tune.suggest import ConcurrencyLimiter

algo = HyperOptSearch(metric="loss",
    mode="min")
name="hypher"

algo = ConcurrencyLimiter(algo, max_concurrent=1)
#os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
scheduler = ASHAScheduler()

parser = argparse.ArgumentParser(description='VAE MNIST Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args, _ = parser.parse_known_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if args.cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': False} if args.cuda else {}

train_loader=  torch.utils.data.DataLoader(
        datasets.STL10('/home/antoine/Projet/data', split="unlabeled", download=True,
                       transform=transforms.ToTensor()),
                           batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
        datasets.STL10('/home/antoine/Projet/data', split="test", transform=transforms.ToTensor(),download=True),
    batch_size=args.batch_size, shuffle=True, **kwargs)

ray.shutdown()
ray.init()

X_id = pin_in_object_store(train_loader) 
Y_id = pin_in_object_store(test_loader)


def trial_name_id(trial):
    return f"{trial.trainable_name}_{trial.trial_id}"
from ray import tune
analysis = tune.run(
train_VAE,
name=name,
scheduler=scheduler,
reuse_actors=False,
search_alg=algo,
verbose=2,
checkpoint_at_end=False,
num_samples=20,
# export_formats=[ExportFormat.MODEL],
            config= {
 "lr":  tune.uniform(1, 8) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "weight_decay":tune.uniform(1, 10)#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": tune.uniform(0, 4)#,1e-4), #*10 et 0
 ,    "b2": tune.uniform(1, 5)#,1e-4), #*10 et 0
 ,    "eps": tune.uniform(2, 10)#,1e-4), #*10 et 0
 ,    "d1": tune.uniform(27648/8, 27648/2)
 ,    "d2": tune.uniform(27648/64, 27648/16)
 ,    "d3": tune.uniform(27648/516, 27648/128)
 ,    "d4": tune.uniform(27648/64, 27648/16)
,    "d5": tune.uniform(27648/8, 27648/2)

                
    },      stop={
        "training_iteration": 3,
    },        metric="loss",
    mode="min",
 #             loggers=[TestLogger])
)
#all_trials = analysis.trials
#checkpoint_paths = [
#    os.path.join(analysis.get_best_checkpoint(t), "checkpoint")
#    for t in all_trials
#] 
#demo_gan(checkpoint_paths)


In [ ]:
from ray.tune import Stopper

class CustomStopper(Stopper):
    def __init__(self):
        self.should_stop = False

    def __call__(self, trial_id, result):
        if not self.should_stop and result['loss'] > 0.48:
          return True
        else:
          return False

    def stop_all(self):
        """Returns whether to stop trials and prevent new ones from starting."""
        return False




In [ ]:
def f_HyperOpt(dataset):
    
    scheduler = AsyncHyperBandScheduler(**experiment_metrics)
    bayesopt = HyperOptSearch(**experiment_metrics,max_concurrent=2)
    stopper = CustomStopper()
    tune.run(dataset, **tune_kwargs , scheduler = scheduler,  name="hyper", search_alg=bayesopt,resources_per_trial={'gpu': 1}, stop = stopper)

In [ ]:
def f_BayesOpt(dataset):
    
    scheduler = AsyncHyperBandScheduler(**experiment_metrics)
    bayesopt = BayesOptSearch(**experiment_metrics,max_concurrent=2)
      stopper = CustomStopper()

    tune.run(dataset, **tune_kwargs , scheduler = scheduler, name="bayes",  search_alg=bayesopt,resources_per_trial={'gpu': 1},stop = stopper)

In [ ]:
def f_AX(dataset):
    
   
    if __name__ == "__main__":
                
        algo = AxSearch(
            max_concurrent=2, #was working with 2
            **experiment_metrics
        )
        scheduler = AsyncHyperBandScheduler(**experiment_metrics)
        stopper = CustomStopper()stop = stopper
        tune.run(
            dataset,       name="ax",
            search_alg=algo,
            scheduler=scheduler,
            **tune_kwargs,resources_per_trial={'gpu': 1},stop = stopper)

        
#        algo = AxSearch(
#            **experiment_metrics
#        )
#        algo = ConcurrencyLimiter(algo, max_concurrent=4)

        
#        scheduler = AsyncHyperBandScheduler()
#        tune.run(
#            dataset,
#            **experiment_metrics,
#            search_alg=algo,
#            scheduler=scheduler,
#            **tune_kwargs)


In [ ]:

# TODO We are interested in multiple Population based algorithms from nevergrad, and certainly not in OnePlusOne. 
def f_NeverGrad(dataset):
    algo = NevergradSearch(
    optimizer=ng.optimizers.CMA
    # space=space,  # If you want to set the space manually
    )
    algo = ConcurrencyLimiter(algo, max_concurrent=2)

    scheduler = AsyncHyperBandScheduler()
    stopper = CustomStopper()
    tune.run(
        dataset,
        **experiment_metrics,
      #  name="nevergrad",
        search_alg=algo, name="ng",
        scheduler=scheduler,
        **tune_kwargs,resources_per_trial={'gpu': 1},stop = stopper)
    

In [ ]:
def f_BOHB(dataset):

    bohb_hyperband = HyperBandForBOHB(
        time_attr="training_iteration",
        max_t=100,
        reduction_factor=2,
        **experiment_metrics)

    bohb_search = TuneBOHB(
        # space=config_space, 
        max_concurrent=2,
        **experiment_metrics)
    stopper = CustomStopper()
    tune.run(
        dataset,
       # config=config, 
        scheduler=bohb_hyperband,name="bohb",
        search_alg=bohb_search,       
         **tune_kwargs,resources_per_trial={'gpu': 1},stop = stopper)
        #num_samples=NUM_TUNED,
       # stop={"training_iteration": 100})
    
    
    

In [ ]:
def f_Random(dataset):
    
    algo = NevergradSearch(
    optimizer=ng.optimizers.RandomSearch,
    # space=space,  # If you want to set the space manually,
    max_concurrent=2
    )
  #  algo = ConcurrencyLimiter(algo)

    scheduler = AsyncHyperBandScheduler()
    stopper = CustomStopper()
    tune.run(
        dataset,
        **experiment_metrics,
      #  name="nevergrad",
        search_alg=algo,   name="random",    
        scheduler=scheduler,
        **tune_kwargs,resources_per_trial={'gpu': 1},stop = stopper)
    

In [ ]:
def f_ZOOpt(dataset):

    dim_dict = {
        "lr": (ValueType.CONTINUOUS, [0, 1], 1e-2),
        "momentum": (ValueType.CONTINUOUS, [0,1, 0.9], 1e-2)
    }

    zoopt_search_config = {
        "parallel_num": 8,  # how many workers to parallel
    }

    
    stopper = CustomStopper()

    zoopt_search = ZOOptSearch(
    algo="Asracos",  # only support Asracos currently
    #dim_dict=dim_dict,
    budget=ITERATIONS,max_concurrent=2,
    #dim_dict=dim_dict,
   #     **zoopt_search_config,
    **experiment_metrics)
    
    scheduler = AsyncHyperBandScheduler(**experiment_metrics)

   
    tune.run(dataset,
 #        config = config,
    search_alg=zoopt_search,
   # num_samples= ITERATIONS,
    scheduler=scheduler,       
    #         paralell_num=4,
    name="zoopt_search", 
              **tune_kwargs,resources_per_trial={'gpu': 1},stop = stopper)
    

In [ ]:
class SentimentRNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_size, n_layers,
             drop_prob, sigmoid , vocab_size,droupout_prob2):
        super(SentimentRNN, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                            dropout=drop_prob, batch_first=True)
        #self.lstm = nn.GRU(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)

        self.dropout = nn.Dropout(droupout_prob2)

        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
        hidden= tuple([each.data for each in hidden])
        batch_size = x.size(0)
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # sigmoid function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels

        # return last sigmoid output and hidden state
        return sig_out, hidden


    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                        weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


class SentimentRNN1(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_size, n_layers,
             drop_prob, sigmoid , vocab_size,droupout_prob2):
        super(SentimentRNN1, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        #self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                        #    dropout=drop_prob, batch_first=True)
        self.lstm = nn.GRU(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)

        self.dropout = nn.Dropout(droupout_prob2)

        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
        hidden = hidden.data
        batch_size = x.size(0)
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # sigmoid function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels

        # return last sigmoid output and hidden state
        return sig_out, hidden


    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                        weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        (a,b) = hidden
        return a
    
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx,sigmoid):

        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)
        
        self.sigmoid = sigmoid
        self.sig = nn.Sigmoid()


    def forward(self, text):

        #text = [sent len, batch size]

       # text = text.permute(1, 0)
        #We want already have batch size, len for sentiment!!!!!
        #text = [batch size, sent len]

        embedded = self.embedding(text)

        #embedded = [batch size, sent len, emb dim]

        embedded = embedded.unsqueeze(1)

        #embedded = [batch size, 1, sent len, emb dim]

        conved = [self.sigmoid(conv(embedded)).squeeze(3) for conv in self.convs]

        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]

        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        #pooled_n = [batch size, n_filters]

        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        sig_out = self.sig(self.fc(cat))

        # reshape to be batch_size first
        sig_out = sig_out.view(50, -1)
        #sig_out = sig_out[:, -1] # get last batch of labels

        # return last sigmoid output and hidden state
        return np.squeeze(sig_out)

In [ ]:
   #https://colab.research.google.com/github/agungsantoso/deep-learning-v2-pytorch/blob/master/sentiment-rnn/Sentiment_RNN_Exercise.ipynb#scrollTo=AVzirwGqpmva
def train_IMDB1(config):
    train_x = np.load('/content/drive/My Drive/train_x.npy')
    train_y = np.load('/content/drive/My Drive/train_y.npy')
    val_x = np.load('/content/drive/My Drive/val_x.npy')
    val_y = np.load('/content/drive/My Drive/val_y.npy')
    test_x = np.load('/content/drive/My Drive/test_x.npy')
    test_y = np.load('/content/drive/My Drive/test_y.npy')
    len_vocab_to_int = 74072
    ## print out the shapes of your resultant feature data
    from torch.utils.data import TensorDataset, DataLoader
    #print(train_x.shape)
    import torch

    # create Tensor datasets
    train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
    valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
    test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

    # dataloaders
    batch_size = 50

    # make sure to SHUFFLE your data
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
    test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)
    # obtain one batch of training data
    dataiter = iter(train_loader)
    sample_x, sample_y = dataiter.next()

    # First checking if GPU is available
    train_on_gpu=torch.cuda.is_available()


    sigmoid_func_uniq = get_sigmoid_func(config.get("sigmoid_func", 0))       
        
        
    # Instantiate the model w/ hyperparams
    vocab_size = len_vocab_to_int + 1 # +1 for zero padding + our word tokens
    output_size = 1
    embedding_dim = int(round(config.get("embedding",64)))
    hidden_dim = int(round(config.get("hidden_dim",64)))
    n_layers =  1+ int( round(config.get("n_layer",1)))
    dropout_2 = config.get("droupout_prob2",0.3);

    FILTER_SIZES = [2,3,4]
    PAD_IDX = 4    


    if(config.get("model",0)<1/3):
        net = SentimentRNN(embedding_dim, hidden_dim, output_size, n_layers,
                           config.get("droupout_prob",0.1),
                           sigmoid_func_uniq, vocab_size,dropout_2)    
    elif(config.get("model",0)<2/3):
        net = SentimentRNN1(embedding_dim, hidden_dim, output_size, n_layers,
                           config.get("droupout_prob",0.1),
                           sigmoid_func_uniq, vocab_size,dropout_2)
    else:
        net = CNN(vocab_size, embedding_dim, hidden_dim, FILTER_SIZES, 
              output_size, config.get("droupout_prob",0.1), PAD_IDX,sigmoid_func_uniq)
    # loss and optimization functions 


        # loss and optimization functions
    lr=10**(-config.get("lr", 1))
    wd=10**(-config.get("weight_decay", 1))

    criterion = nn.L1Loss()
    
    
    if(config.get("adam",0)>0.5):
        optimizer = torch.optim.Adam(net.parameters(), lr=lr, 
           weight_decay=wd, 
                                         amsgrad=True)
    else: 
        optimizer = adabelief_pytorch.AdaBelief(net.parameters(), lr=lr, 
                              weight_decay=wd)
    


        # training params

    epochs = ITERATIONS # 3-4 is approx where I noticed the validation loss stop decreasing

    counter = 0
    print_every = 10
    clip=1 # gradient clipping



    # move model to GPU, if available
    if(train_on_gpu):
        net.cuda()

    net.train()
    # train for some number of epochs
    
    
    def trainlstm():
     #   criterion = nn.BCELoss()

         # initialize hidden state
        h = net.init_hidden(batch_size)

        # batch loop
        for inputs, labels in train_loader:

            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            
            
            
            h = tuple([each.data for each in h])
            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output.squeeze(), labels.float())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()

    def traingru():
        h = net.init_hidden(batch_size)
     #   criterion = nn.BCELoss()

        # batch loop
        for inputs, labels in train_loader:

            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            
            
            
           # h = tuple([each.data for each in h])
            h = h.data
            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output.squeeze(), labels.float())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()
       
    
    def train_cnn():
       # criterion = nn.BCELoss()

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        net.train()
        #for (data, target) in train_loader:
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            # We set this just for the example to run quickly.
            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()


            optimizer.zero_grad()
            output = net(inputs)

            loss = criterion(output.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
    
    def test_lstm(x):
       
        val_h = net.init_hidden(batch_size)
        val_losses = []
        net.eval()
        for inputs, labels in x:

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history

            val_h = tuple([each.data for each in val_h])
            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()

            output, val_h = net(inputs, val_h)
            val_loss = criterion(output.squeeze(), labels.float())

            val_losses.append(val_loss.item())

        net.train()
        return np.mean(val_losses) 

            
    def test_gru(x):
        
        val_h = net.init_hidden(batch_size)
        val_losses = []
        net.eval()
        for inputs, labels in x:

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history

            #val_h = tuple([each.data for each in val_h])
            val_h = val_h.data
            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()

            output, val_h = net(inputs, val_h)
            val_loss = criterion(output.squeeze(), labels.float())

            val_losses.append(val_loss.item())

        net.train()
    
        return np.mean(val_losses) 
    
    def test_cnn(x):
        val_losses = []
        net.eval()
        for inputs, labels in x:
            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()
            output = net(inputs)
            val_loss = criterion(output.squeeze(), labels.float())

            val_losses.append(val_loss.item())

         
        net.train()
    
        return np.mean(val_losses)     
            

    for e in range(epochs):

       
        model = config.get("model",0)
        if(model<1/3) :
            trainlstm()
            loss=test_lstm(valid_loader)
            acc = test_lstm(test_loader)

        elif(model<2/3):
            traingru()
            loss=test_gru(valid_loader)
            acc = test_gru(test_loader)
        else:
            train_cnn()
            loss=test_cnn(valid_loader)
            acc = test_cnn(test_loader)
               
        tune.report(loss=loss,mean_accuracy=acc)



In [ ]:
#Configs
parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke_test", action="store_false", help="Finish not quickly for testing")
args, _ = parser.parse_known_args()          
        
    
    
#experiment_metrics = dict(metric="mean_accuracy", mode="max")
experiment_metrics = dict(metric="loss", mode="min")


ITERATIONS = 4
NUM_TUNED=  128
    

#i is in [0;1]
#We want all values between 0 and 1
def get_sigmoid_func(i):
    if(i<0.33):
        return nn.ReLU()
    elif(i<0.67):
        return nn.Tanh()
    else:
        return nn.Sigmoid()





tune_kwargs = {
    "num_samples": NUM_TUNED if args.smoke_test else NUM_TUNED,
    "config": {
    "steps": ITERATIONS,  
     "lr": tune.uniform(2, 7 ) #will be x->10**-x
       ,  "embedding": tune.uniform(200, 1024) #Pas bien #Different embeddings, fine tune ou pas
      ,"droupout_prob2":tune.uniform(0.3,0.6)
         ,"weight_decay": tune.uniform(2,7) #will be x->10**-x
      ,  "sigmoid_func":tune.uniform(0,1),
        "hidden_dim":tune.uniform(32.,512.),
        "n_layer":tune.uniform(1,5),
        "droupout_prob":tune.uniform(0.3,0.6)  
     ,   "adam":tune.uniform(0,1)
      ,  "model":tune.uniform(0,1)
    }
}


#x_all = [train_IMDB,  train_TREC, train_boston, train_diabetes, train_mnist, train_fashion_mnist]

#f_HyperOpt(train_IMDB1)

#f_HyperOpt(train_IMDB1) 

In [ ]:
for i in range(0,1):
    for j in range(0,1):
        x = train_IMDB1
        f_BayesOpt(x)
        #f_AX(x)
        #f_NeverGrad(x)
        #f_BOHB(x)
        #f_Random(x)
        #f_ZOOpt(x)
        print("all worked with " + str(x)+  " !")